In [1]:
%pip install docker pymongo pandas

You should consider upgrading via the '/home/david/.pyenv/versions/3.8.6/envs/python-data-prep/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import docker

docker_client = docker.from_env()
MONGO_PORT = 27018

mongo_container = docker_client.containers.run(
    'mongo:latest',
    detach=True,
    name='mongo-data-prep',
    remove=True,
    ports={'27017/tcp': MONGO_PORT},
    mem_limit='2G',
    volumes={
        os.path.join(os.getcwd(), 'db'): {'bind': '/data/db', 'mode': 'rw'},
    }
)

In [3]:
from pymongo import MongoClient
mongo_client = MongoClient('127.0.0.1', MONGO_PORT)
db = mongo_client['data-prep']

In [4]:
sales_col = db.get_collection('sales')
sales_col.drop()

In [10]:
from pathlib import Path
import json
sales_data = json.loads(Path('/home/david/dev/toucan-toco/python-data-prep/data/sales.json').read_text())
sales_col.insert_many(sales_data['data'])

Simulating a weaverbird pipeline:

In [13]:
import pandas as pd

In [27]:
'''
{
    "domain": "sales",
    "name": "domain"
},
'''
def get_domain(domain):
    return pd.DataFrame(db.get_collection(domain).find({}))
df = get_domain('sales')

In [28]:
'''
{
    "name": "filter",
    "condition": {
      "column": "Payment_Type",
      "operator": "eq",
      "value": "Mastercard"
    }
}
'''
def filter(df, column, value):
    return df[df[column] == value]
df1 = filter(df, 'Payment_Type', 'Mastercard')

In [31]:
%%time

df = get_domain('sales')
filter(df, 'Payment_Type', 'Mastercard')

CPU times: user 5.95 ms, sys: 0 ns, total: 5.95 ms
Wall time: 5.43 ms


,_id,Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude
1,5f903839acd166c1735d9216,1/5/09 4:10,Product1,1200,Mastercard,Nicola,Roodepoort,Gauteng,South Africa,1/5/09 2:33,1/7/09 5:13,-26.166667,27.866667
2,5f903839acd166c1735d9217,1/4/09 13:17,Product1,1200,Mastercard,Renee Elisabeth,Tel Aviv,Tel Aviv,Israel,1/4/09 13:03,1/4/09 22:10,32.066667,34.766667
3,5f903839acd166c1735d9218,1/2/09 6:17,Product1,1200,Mastercard,carolina,Basildon,England,United Kingdom,1/2/09 6:00,1/2/09 6:08,51.500000,-1.116667
6,5f903839acd166c1735d921b,1/5/09 8:58,Product2,3600,Mastercard,Marcia,Telgte,Nordrhein-Westfalen,Germany,9/1/08 3:39,1/14/09 2:07,52.333333,7.900000
9,5f903839acd166c1735d921e,1/15/09 12:54,Product1,1200,Mastercard,Annelies,Ile-Perrot,Quebec,Canada,1/15/09 12:22,1/16/09 7:53,45.400000,-73.933333
15,5f903839acd166c1735d9224,1/12/09 15:12,Product1,1200,Mastercard,David,Deptford,NJ,United States,1/12/09 14:07,1/19/09 3:47,39.838060,-75.153060
16,5f903839acd166c1735d9225,1/19/09 16:10,Product1,1200,Mastercard,Frank,Old Greenwich,CT,United States,1/19/09 15:31,1/19/09 16:00,41.022780,-73.565280
17,5f903839acd166c1735d9226,1/20/09 6:03,Product1,1200,Mastercard,Andrea,Shreveport,LA,United States,1/20/09 5:13,1/20/09 7:15,32.525000,-93.750000
26,5f903839acd166c1735d922f,1/4/09 9:54,Product1,1200,Mastercard,AMY,The Woodlands,TX,United States,12/30/08 20:41,1/25/09 18:23,30.157780,-95.489170
27,5f903839acd166c1735d9230,1/22/09 15:32,Product1,1200,Mastercard,Tara,Killiney,Dublin,Ireland,2/27/07 11:35,1/26/09 4:32,53.252222,-6.112500
